# Notebook for preprocessing Persian Wikipedia dataset

In [ ]:
!pip install pebble hf_transfer -q

### Initialize configuration

In [1]:
import yaml

config_path = "Configs/config_fa.yml"  # Persian config
config = yaml.safe_load(open(config_path))

### Initialize phonemizer and tokenizer

In [2]:
# !pip install git+https://git@github.com/SadeghKrmi/pernorm.git
# !pip install git+https://git@github.com/SadeghKrmi/zirneshane.git
# !pip install git+https://git@github.com/SadeghKrmi/vaguye.git
# !pip install git+https://git@github.com/SadeghKrmi/hamnevise.git

In [2]:
from phonemize_fa import phonemize

In [3]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(config['dataset_params']['tokenizer'])  # Persian BERT tokenizer

### Load Persian Wikipedia dataset

In [4]:
from load_persian_dataset import load_persian_wikipedia
dataset = load_persian_wikipedia("./datasets/wikipedia-fa-norm-hamnevise-zirneshane.txt")

Loading Persian Wikipedia from ./datasets/wikipedia-fa-norm-hamnevise-zirneshane.txt...
Loaded 1368002 lines from Persian Wikipedia
Created dataset with 1368002 examples


### Setup multiprocessing for dataset processing

In [5]:
root_directory = "./datasets/wiki_phoneme_fa"  # set up root directory for multiprocessor processing

In [7]:
import os
num_shards = 5000

def process_shard(i):
    directory = root_directory + "/shard_" + str(i)
    if os.path.exists(directory):
        print("Shard %d already exists!" % i)
        return
    print('Processing shard %d ...' % i)
    shard = dataset.shard(num_shards=num_shards, index=i)
    processed_dataset = shard.map(lambda t: phonemize(t['text'], tokenizer), remove_columns=['text'])
    if not os.path.exists(directory):
        os.makedirs(directory)
    processed_dataset.save_to_disk(directory)

In [8]:
from pebble import ProcessPool
from concurrent.futures import TimeoutError

#### Note: You may need to run the following cell multiple times to process all shards because some will fail. Depending on how fast you process each shard, you may need to change the timeout to a longer value.

In [ ]:
max_workers = 26  # change this to the number of CPU cores your machine has

with ProcessPool(max_workers=max_workers) as pool:
    # Increased timeout to 300s (5 mins) to be safe
    future = pool.map(process_shard, range(num_shards), timeout=2000)
    for result in future.result():
        pass


Shard 0 already exists!
Processing shard 1 ...
Shard 2 already exists!
Shard 3 already exists!
Shard 4 already exists!
Shard 6 already exists!Shard 5 already exists!Shard 7 already exists!Shard 8 already exists!Shard 9 already exists!



Shard 10 already exists!
Shard 11 already exists!Shard 12 already exists!
Shard 13 already exists!

Shard 14 already exists!Shard 15 already exists!
Shard 17 already exists!
Shard 16 already exists!

Shard 19 already exists!
Shard 18 already exists!Shard 20 already exists!
Shard 21 already exists!

Shard 22 already exists!Shard 23 already exists!


Shard 24 already exists!Shard 25 already exists!Shard 26 already exists!


Shard 28 already exists!Shard 29 already exists!Shard 30 already exists!Shard 31 already exists!
Shard 32 already exists!
Shard 33 already exists!



Shard 27 already exists!
Shard 34 already exists!
Shard 35 already exists!
Shard 36 already exists!Shard 41 already exists!Shard 45 already exists!Shard 42 already exists!Shard 49 alread

Map:   0%|          | 0/274 [00:00<?, ? examples/s]




Shard 912 already exists!

Shard 913 already exists!Shard 914 already exists!
Shard 915 already exists!Shard 917 already exists!Shard 918 already exists!
Shard 916 already exists!Shard 921 already exists!

Shard 919 already exists!Shard 920 already exists!Shard 922 already exists!Shard 923 already exists!
Shard 924 already exists!
Shard 925 already exists!Shard 926 already exists!Shard 927 already exists!Shard 928 already exists!

Shard 929 already exists!
Shard 930 already exists!

Shard 931 already exists!
Shard 932 already exists!

Shard 933 already exists!





Shard 934 already exists!Shard 935 already exists!

Shard 936 already exists!
Shard 937 already exists!
Shard 938 already exists!
Shard 939 already exists!Shard 940 already exists!

Shard 941 already exists!Shard 942 already exists!Shard 943 already exists!Shard 944 already exists!Shard 945 already exists!Shard 946 already exists!Shard 947 already exists!







Shard 954 already exists!Shard 948 already exists!Shard 952 

### Collect all shards to form the processed dataset

In [7]:
import os
from datasets import load_from_disk, concatenate_datasets

output = [dI for dI in os.listdir(root_directory) if os.path.isdir(os.path.join(root_directory,dI))]
datasets = []
for o in output:
    directory = root_directory + "/" + o
    try:
        shard = load_from_disk(directory)
        datasets.append(shard)
        print("%s loaded" % o)
    except:
        continue

shard_4997 loaded
shard_4996 loaded
shard_4992 loaded
shard_4999 loaded
shard_4995 loaded
shard_4998 loaded
shard_4993 loaded
shard_4989 loaded
shard_4988 loaded
shard_4990 loaded
shard_4994 loaded
shard_4987 loaded
shard_4991 loaded
shard_4986 loaded
shard_4985 loaded
shard_4982 loaded
shard_4983 loaded
shard_4981 loaded
shard_4984 loaded
shard_4979 loaded
shard_4980 loaded
shard_4977 loaded
shard_4978 loaded
shard_4976 loaded
shard_4975 loaded
shard_4969 loaded
shard_4971 loaded
shard_4968 loaded
shard_4972 loaded
shard_4974 loaded
shard_4970 loaded
shard_4973 loaded
shard_4967 loaded
shard_4964 loaded
shard_4966 loaded
shard_4963 loaded
shard_4965 loaded
shard_4962 loaded
shard_4961 loaded
shard_4960 loaded
shard_4959 loaded
shard_4958 loaded
shard_4955 loaded
shard_4957 loaded
shard_4938 loaded
shard_4956 loaded
shard_4954 loaded
shard_4952 loaded
shard_4953 loaded
shard_4951 loaded
shard_4950 loaded
shard_4939 loaded
shard_4949 loaded
shard_4946 loaded
shard_4947 loaded
shard_4945

In [8]:
dataset = concatenate_datasets(datasets)
dataset.save_to_disk(config['data_folder'])
print('Dataset saved to %s' % config['data_folder'])

Saving the dataset (0/2 shards):   0%|          | 0/1366906 [00:00<?, ? examples/s]

Dataset saved to wikipedia_fa.processed


In [10]:
# # load from disk

# from datasets import load_from_disk

# dataset = load_from_disk(config['data_folder'])
# print("Dataset loaded from %s" % config['data_folder'])


Dataset loaded from wikipedia_fa.processed


In [11]:
# check the dataset size
dataset

Dataset({
    features: ['input_ids', 'phonemes'],
    num_rows: 1366906
})

In [ ]:
# print(dataset[1]['phonemes'])

### Generate token maps for Persian vocabulary
We need to create mappings from BERT token IDs to reduced vocabulary for efficient training.

In [12]:
from simple_loader import FilePathDataset, build_dataloader

file_data = FilePathDataset(dataset)
loader = build_dataloader(file_data, num_workers=16, batch_size=128)

In [13]:
special_token = config['dataset_params']['word_separator']

In [14]:
# get all unique tokens in the entire dataset

from tqdm import tqdm

unique_index = [special_token]
for _, batch in enumerate(tqdm(loader)):
    unique_index.extend(batch)
    unique_index = list(set(unique_index))

100%|██████████| 10678/10678 [01:02<00:00, 170.08it/s]


In [15]:
# For Persian BERT, we don't need to lowercase (parsbert is already uncased)
# Just create direct mapping

lower_tokens = list(set(unique_index))

In [16]:
# create the mapping for Persian tokens

token_maps = {}
for t in tqdm(unique_index):
    word = tokenizer.decode([t])
    token_maps[t] = {'word': word, 'token': lower_tokens.index(t)}

100%|██████████| 74967/74967 [00:31<00:00, 2405.36it/s] 


In [17]:
import pickle
with open(config['dataset_params']['token_maps'], 'wb') as handle:
    pickle.dump(token_maps, handle)
print('Token mapper saved to %s' % config['dataset_params']['token_maps'])

Token mapper saved to token_fa_maps.pkl


### Test the dataset with dataloader

In [18]:
from dataloader import build_dataloader

train_loader = build_dataloader(dataset, batch_size=32, num_workers=0, dataset_config=config['dataset_params'])

177


In [19]:
_, (words, labels, phonemes, input_lengths, masked_indices) = next(enumerate(train_loader))